
### **Setup & Dependencies**

In [7]:
# @title  ###Install **pyrosetta** (~3 min, run this only once)

#@markdown Rosetta is now [more open](https://x.com/RosettaCommons/status/1766164143403364808). Non-commercial users no longer need to explicitly sign up for a license to access Rosetta. **A license is still needed for commercial use.**

!pip install pyrosettacolabsetup --quiet

import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta(serialization=True)
import pyrosetta; # import the pyrosetta package to access functionality
import pyrosetta.distributed.io as io
pyrosetta.init("-corrections::beta_nov16 -mute all")
from google.colab import files



Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels.serialization...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels.serialization//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels.serialization/pyrosetta-2025.41+release.de3cc17d50-cp312-cp312-linux_x86_64.whl


┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR C

In [2]:
#@title setup **RFdiffusion** (~3min - run this only once )
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
  os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
  # 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
  # 25Aug2025: updating dgi install to work with latest pytorch
  os.system("pip install --no-dependencies dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html")
  os.system("pip install --no-dependencies e3nn==0.5.5 opt_einsum_fx")
  os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt","Complex_beta_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    if not os.path.isfile(f"{pdb_code}.pdb1"):
      os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz")
      os.system(f"gunzip {pdb_code}.pdb1.gz")
    return f"{pdb_code}.pdb1"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

def run_ananas(pdb_str, path, sym=None):
  pdb_filename = f"outputs/{path}/ananas_input.pdb"
  out_filename = f"outputs/{path}/ananas.json"
  with open(pdb_filename,"w") as handle:
    handle.write(pdb_str)

  cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
  if sym is None: os.system(cmd)
  else: os.system(f"{cmd} {sym}")

  # parse results
  try:
    out = json.loads(open(out_filename,"r").read())
    results,AU = out[0], out[-1]["AU"]
    group = AU["group"]
    chains = AU["chain names"]
    rmsd = results["Average_RMSD"]
    print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

    C = np.array(results['transforms'][0]['CENTER'])
    A = [np.array(t["AXIS"]) for t in results['transforms']]

    # apply symmetry and filter to the asymmetric unit
    new_lines = []
    for line in pdb_str.split("\n"):
      if line.startswith("ATOM"):
        chain = line[21:22]
        if chain in chains:
          x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
          if group[0] == "c":
            x = sym_it(x,C,A[0])
          if group[0] == "d":
            x = sym_it(x,C,A[1],A[0])
          coord_str = "".join(["{:8.3f}".format(a) for a in x])
          new_lines.append(line[:30]+coord_str+line[54:])
      else:
        new_lines.append(line)
    return results, "\n".join(new_lines)

  except:
    return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

  def run_command_and_get_pid(command):
    pid_file = '/dev/shm/pid'
    os.system(f'nohup {command} & echo $! > {pid_file}')
    with open(pid_file, 'r') as f:
      pid = int(f.read().strip())
    os.remove(pid_file)
    return pid
  def is_process_running(pid):
    try:
      os.kill(pid, 0)
    except OSError:
      return False
    else:
      return True

  run_output = widgets.Output()
  progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
  display(widgets.VBox([progress, run_output]))

  # clear previous run
  for n in range(steps):
    if os.path.isfile(f"/dev/shm/{n}.pdb"):
      os.remove(f"/dev/shm/{n}.pdb")

  pid = run_command_and_get_pid(command)
  try:
    fail = False
    for _ in range(num_designs):

      # for each step check if output generated
      for n in range(steps):
        wait = True
        while wait and not fail:
          time.sleep(0.1)
          if os.path.isfile(f"/dev/shm/{n}.pdb"):
            pdb_str = open(f"/dev/shm/{n}.pdb").read()
            if pdb_str[-3:] == "TER":
              wait = False
            elif not is_process_running(pid):
              fail = True
          elif not is_process_running(pid):
            fail = True

        if fail:
          progress.bar_style = 'danger'
          progress.description = "failed"
          break

        else:
          progress.value = (n+1) / steps
          if visual != "none":
            with run_output:
              run_output.clear_output(wait=True)
              if visual == "image":
                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                fig = plt.figure()
                fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1)
                plt.show()
              if visual == "interactive":
                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                view.addModel(pdb_str,'pdb')
                view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                view.zoomTo()
                view.show()
        if os.path.exists(f"/dev/shm/{n}.pdb"):
          os.remove(f"/dev/shm/{n}.pdb")
      if fail:
        progress.bar_style = 'danger'
        progress.description = "failed"
        break

    while is_process_running(pid):
      time.sleep(0.1)

  except KeyboardInterrupt:
    os.kill(pid, signal.SIGTERM)
    progress.bar_style = 'danger'
    progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, partial_T="auto",
                  num_designs=1, use_beta_model=False, visual="none"):

  full_path = f"outputs/{path}"
  os.makedirs(full_path, exist_ok=True)
  opts = [f"inference.output_prefix={full_path}",
          f"inference.num_designs={num_designs}"]

  if chains == "": chains = None

  # determine symmetry type
  if symmetry in ["auto","cyclic","dihedral"]:
    if symmetry == "auto":
      sym, copies = None, 1
    else:
      sym, copies = {"cyclic":(f"c{order}",order),
                     "dihedral":(f"d{order}",order*2)}[symmetry]
  else:
    symmetry = None
    sym, copies = None, 1

  # determine mode
  contigs = contigs.replace(","," ").replace(":"," ").split()
  is_fixed, is_free = False, False
  fixed_chains = []
  for contig in contigs:
    for x in contig.split("/"):
      a = x.split("-")[0]
      if a[0].isalpha():
        is_fixed = True
        if a[0] not in fixed_chains:
          fixed_chains.append(a[0])
      if a.isnumeric():
        is_free = True
  if len(contigs) == 0 or not is_free:
    mode = "partial"
  elif is_fixed:
    mode = "fixed"
  else:
    mode = "free"

  # fix input contigs
  if mode in ["partial","fixed"]:
    pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
    if symmetry == "auto":
      a, pdb_str = run_ananas(pdb_str, path)
      if a is None:
        print(f'ERROR: no symmetry detected')
        symmetry = None
        sym, copies = None, 1
      else:
        if a["group"][0] == "c":
          symmetry = "cyclic"
          sym, copies = a["group"], int(a["group"][1:])
        elif a["group"][0] == "d":
          symmetry = "dihedral"
          sym, copies = a["group"], 2 * int(a["group"][1:])
        else:
          print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
          symmetry = None
          sym, copies = None, 1

    elif mode == "fixed":
      pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

    pdb_filename = f"{full_path}/input.pdb"
    with open(pdb_filename, "w") as handle:
      handle.write(pdb_str)

    parsed_pdb = parse_pdb(pdb_filename)
    opts.append(f"inference.input_pdb={pdb_filename}")
    if mode in ["partial"]:
      if partial_T == "auto":
        iterations = int(80 * (iterations / 200))
      else:
        iterations = int(partial_T)
      opts.append(f"diffuser.partial_T={iterations}")
      contigs = fix_partial_contigs(contigs, parsed_pdb)
    else:
      opts.append(f"diffuser.T={iterations}")
      contigs = fix_contigs(contigs, parsed_pdb)
  else:
    opts.append(f"diffuser.T={iterations}")
    parsed_pdb = None
    contigs = fix_contigs(contigs, parsed_pdb)

  if hotspot is not None and hotspot != "":
    hotspot = ",".join(hotspot.replace(","," ").split())
    opts.append(f"ppi.hotspot_res='[{hotspot}]'")

  # setup symmetry
  if sym is not None:
    sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
    if add_potential:
      sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                   "potentials.olig_intra_all=True","potentials.olig_inter_all=True",
                   "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
    opts = sym_opts + opts
    contigs = sum([contigs] * copies,[])

  opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
  opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
  if use_beta_model:
    opts += ["inference.ckpt_override_path=./RFdiffusion/models/Complex_beta_ckpt.pt"]

  print("mode:", mode)
  print("output:", full_path)
  print("contigs:", contigs)

  opts_str = " ".join(opts)
  cmd = f"./RFdiffusion/run_inference.py {opts_str}"
  print(cmd)

  # RUN
  run(cmd, iterations, num_designs, visual=visual)

  # fix pdbs
  for n in range(num_designs):
    pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb",
            f"outputs/traj/{path}_{n}_Xt-1_traj.pdb",
            f"{full_path}_{n}.pdb"]
    for pdb in pdbs:
      with open(pdb,"r") as handle: pdb_str = handle.read()
      with open(pdb,"w") as handle: handle.write(fix_pdb(pdb_str, contigs))

  return contigs, copies

installing RFdiffusion...
installing ColabDesign...
downloading RFdiffusion params...


/content/RFdiffusion/diffusion.py:276: SyntaxWarning: invalid escape sequence '\s'
  Extract \sigma(t) corresponding to chosen sigma schedule.
/content/RFdiffusion/diffusion.py:303: SyntaxWarning: invalid escape sequence '\i'
  sigma(t)^2 := \int_0^t g(s)^2 ds,
/content/RFdiffusion/Track_module.py:241: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


CPU times: user 9.43 s, sys: 1.1 s, total: 10.5 s
Wall time: 2min 17s


In [8]:
# @title Setups and Imports 🔗
# @markdown The notebook must be connected to Google Drive, so please run this cell to mount your Google Drive
# @markdown Please run this cell for **once** to setup tools
# @markdown
# @markdown *********

!pip install py3Dmol
!pip install biopython
import py3Dmol
from IPython.display import HTML, display, Markdown
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
from google.colab import drive
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
import requests
import time

# Helper to get PDB
def get_pdb(pdb_code="", outfolder=""):
  if outfolder != "":
    os.system(f"mkdir -p {outfolder}")
    wget_suff = f'-P {outfolder}'
  else:
    wget_suff = ""
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    p = os.path.join(outfolder, "tmp.pdb")
    with open(p, "wb") as out:
      out.write(pdb_string)
    return p
  elif os.path.isfile(os.path.join(outfolder, pdb_code)):
    return os.path.join(outfolder, pdb_code)
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb {wget_suff}")
    p = os.path.join(outfolder, f"{pdb_code}.pdb")
    assert os.path.isfile(p), "Specified PDB id with length 4: Could not be downloaded from PDB database"
    return p
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb {wget_suff}")
    p = os.path.join(outfolder, f"AF-{pdb_code}-F1-model_v3.pdb")
    assert os.path.isfile(p), "Specified PDB id length above 4: Could not be downloaded from Alphafold DB"
    return p

def plot_pdb(pdb, color:str="chain", show_sticks:bool=True):
  import py3Dmol
  from string import ascii_uppercase, ascii_lowercase
  alphabet_list = list(ascii_uppercase+ascii_lowercase)
  from pyrosetta import pose_from_file

  pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]
  hbondCutoff = 4.0
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(pdb, 'r').read()
  view.addModel(pdb_str, 'pdb', {'hbondCutoff':hbondCutoff})

  pose = pose_from_file(pdb)
  n_chains = len(set([pose.pdb_info().chain(i) for i in range(1, len(pose.sequence()))]))
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50, 'max':100}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for chain, color in zip(alphabet_list, pymol_color_list):
      view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  else:
    view.setStyle({"cartoon": {'color': 'orange'}})

  if show_sticks:
    view.addStyle({'stick':{'colorscheme': "GrayCarbon"}})
  view.zoomTo()
  view.show()

In [12]:
#@title Download pdbs and visualization
# @markdown Enter PDB ID that you would like to visualize

pdb ='1ubq' #@param {type:"string"}
#@markdown - four letter PDB code will automatically downloaded from the PDB.
#@markdown - providing a UniProt ID will download the AF2 model from the [AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/)
#@markdown - leave blank to get an upload prompt and use a pdb from your system
pdb_path = get_pdb(pdb)

colorscheme = "rainbow" #@param ["rainbow", "chain", "pLDDT"]
show_sidechains = True #@param {type:"boolean"}
assert os.path.isfile(pdb_path), f"Specified PDB file not found: {f}"
plot_pdb(pdb_path, colorscheme, show_sidechains)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
%%time
import os, time, signal
import sys, random, string, re
#@title Run **RFdiffusion** to generate a backbone of binders
#@markdown ###<u>RFdiffusion Settings</u>
#@markdown
#@markdown Write below the name of the output folder

name = "test" #@param {type:"string"}
#@markdown Define binder information to create (e.g., chain ID, specific range, length of binder - check buttom of the notebook how to define contigs)
contigs = "100" #@param {type:"string"}
#@markdown Write the PDB ID name which will be used as target
pdb = "" #@param {type:"string"}
#@markdown Write iteration number (default is 50)
iterations = 50 #@param ["25", "50", "100", "150", "200"] {type:"raw"}
#@markdown Write below epitope residues as number which will be interaction residues for binder (e.g. A90,A91,A93 (please do NOT use space between them))
hotspot = "" #@param {type:"string"}
#@markdown Write below how many design you would like to create. For our case, the default is 2
num_designs = 2 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
#@markdown Defines how the generated structure will be displayed (none, static image, or interactive 3D viewer).
visual = "none" #@param ["none", "image", "interactive"]
#@markdown ---
#@markdown ####<u>**symmetry** settings </u>
#@markdown Specifies whether to apply symmetry in the design (none, auto-detect, cyclic, or dihedral)
symmetry = "none" #@param ["none", "auto", "cyclic", "dihedral"]
#@markdown Sets the number of repeating units for symmetry (used only if symmetry is enabled)
order = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"] {type:"raw"}
#@markdown Limits symmetry detection to specific chains from the input PDB
chains = "" #@param {type:"string"}
#@markdown Adds repulsive potential to reduce clashes between chains during generation
add_potential = False #@param {type:"boolean"}
##@markdown - `symmetry='auto'` enables automatic symmetry dectection with [AnAnaS](https://team.inria.fr/nano-d/software/ananas/).
##@markdown - `chains="A,B"` filter PDB input to these chains (may help auto-symm detector)
##@markdown - `add_potential` to discourage clashes between chains
#@markdown ---
#@markdown ####<u>*Advanced Settings*</u>
#@markdown
#@markdown specify number of noising steps (only used for the partial diffusion protocol)
partial_T = "auto" # @param ["auto", "10", "20", "40", "60", "80"]
#@markdown If you are seeing lots of helices, switch to the "beta" params for a better SSE balance.
use_beta_model = False #@param {type:"boolean"}

# determine where to save
path = name
while os.path.exists(f"outputs/{path}_0.pdb"):
  path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "visual":visual}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')

contigs, copies = run_diffusion(**flags)

mode: free
output: outputs/test
contigs: ['100-100']
./RFdiffusion/run_inference.py inference.output_prefix=outputs/test inference.num_designs=2 diffuser.T=50 'contigmap.contigs=[100-100]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'


CPU times: user 212 ms, sys: 65.4 ms, total: 277 ms
Wall time: 1min 43s


In [4]:
#@title Display 3D structure of backbones {run: "auto"}
animate = "movie" #@param ["none", "movie", "interactive"]
color = "chain" #@param ["rainbow", "chain", "plddt"]
denoise = True
dpi = 100 #@param ["100", "200", "400"] {type:"raw"}
from colabdesign.shared.plot import pymol_color_list
from colabdesign.rf.utils import get_ca, get_Ls, make_animation
from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def plot_pdb(num=0):
  if denoise:
    pdb_traj = f"outputs/traj/{path}_{num}_pX0_traj.pdb"
  else:
    pdb_traj = f"outputs/traj/{path}_{num}_Xt-1_traj.pdb"
  if animate in ["none","interactive"]:
    hbondCutoff = 4.0
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if animate == "interactive":
      pdb_str = open(pdb_traj,'r').read()
      view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    else:
      pdb = f"outputs/{path}_{num}.pdb"
      pdb_str = open(pdb,'r').read()
      view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    if color == "rainbow":
      view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
      for n,chain,c in zip(range(len(contigs)),
                              alphabet_list,
                              pymol_color_list):
          view.setStyle({'chain':chain},{'cartoon': {'color':c}})
    else:
      view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
    view.zoomTo()
    if animate == "interactive":
      view.animate({'loop': 'backAndForth'})
    view.show()
  else:
    Ls = get_Ls(contigs)
    xyz, bfact = get_ca(pdb_traj, get_bfact=True)
    xyz = xyz.reshape((-1,sum(Ls),3))[::-1]
    bfact = bfact.reshape((-1,sum(Ls)))[::-1]
    if color == "chain":
      display(HTML(make_animation(xyz, Ls=Ls, dpi=dpi, ref=-1)))
    elif color == "rainbow":
      display(HTML(make_animation(xyz, dpi=dpi, ref=-1)))
    else:
      display(HTML(make_animation(xyz, plddt=bfact*100, dpi=dpi, ref=-1)))


if num_designs > 1:
  output = widgets.Output()
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
      options=[(f'{k}',k) for k in range(num_designs)],
      value=0, description='design:',
  )
  dropdown.observe(on_change)
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()

In [5]:
%%time
import os, time, signal
import sys, random, string, re
#@title 2.2. Run **ProteinMPNN** to generate a sequence and **AlphaFold** to validate 🏃🏻‍♀️‍➡️
#@markdown ###<u>ProteinMPNN Settings</u>
#@markdown Write below number of sequences that you would like to design
num_seqs = 3 #@param ["1", "2", "3", "4", "8", "16", "32", "64"] {type:"raw"}
#@markdown Select temperature of sample. It is to control diversity of sampled sequences (higher = more diverse)
mpnn_sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
#@markdown Write here the residue letter code of the amino acid that is not wanted to be used in the design, or that is wanted to be removed or changed. Defaul is C (do not use [C]ysteines)
rm_aa = "C" #@param {type:"string"}
#@markdown Option for usage of [solubleMPNN](https://www.biorxiv.org/content/10.1101/2023.05.09.540044v2). ProteinMPNN tries to avoid using hydrophobic residues on the surface. Instead, it favors polar and charged amino acids, which will increase solubility. It uses weights trained only on soluble proteins. (Keep it selected)
use_solubleMPNN = True #@param {type:"boolean"}
#@markdown -----
#@markdown ###<u>AlphaFold2 Settings</u>
#@markdown Use when there is a starting structure to be used in iterations which [increases design success rates](https://doi.org/10.1038/s41467-023-38328-5).
#@markdown (For binder design, keep it selected)
initial_guess = True #@param {type:"boolean"}
#@markdown The number of cycles is the number of times AlphaFold2 will reprocess its structure and improve its prediction
num_recycles = 3 #@param ["0", "1", "2", "3", "6", "12"] {type:"raw"}
#@markdown Use AlphaFold Multimer v3 params for prediction (Keep it selected)
use_multimer = True #@param {type:"boolean"}

if not os.path.isfile("params/done.txt"):
  print("downloading AlphaFold params...")
  while not os.path.isfile("params/done.txt"):
    time.sleep(5)

contigs_str = ":".join(contigs)
opts = [f"--pdb=outputs/{path}_0.pdb",
        f"--loc=outputs/{path}",
        f"--contig={contigs_str}",
        f"--copies={copies}",
        f"--num_seqs={num_seqs}",
        f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}",
        f"--mpnn_sampling_temp={mpnn_sampling_temp}",
        f"--num_designs={num_designs}"]
if initial_guess: opts.append("--initial_guess")
if use_multimer: opts.append("--use_multimer")
if use_solubleMPNN: opts.append("--use_soluble")
opts = ' '.join(opts)
!python colabdesign/rf/designability_test.py {opts}

{'pdb':'outputs/test_0.pdb','loc':'outputs/test','contigs':'100-100','copies':1,'num_seqs':3,'initial_guess':True,'use_multimer':True,'use_soluble':True,'num_recycles':3,'rm_aa':'C','num_designs':2,'mpnn_sampling_temp':0.1}
protocol=fixbb
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.036 plddt:0.975 ptm:0.890 pae:1.910 rmsd:0.633 SELEERLKELFKRYEEAVERALALLERLEELRPEEREELEELRRELEEELEELKKKVEEVLKSGDEEEAKKLIEELEEKTEEVEELIEELEEKLRELEEE
design:0 n:1 mpnn:0.968 plddt:0.958 ptm:0.861 pae:2.596 rmsd:0.767 MELEKELEEKFKEYKEKAEEVLKLLEELKKKIPEKKEELEAKKKEIEKKIKELEERKKKVLEKGDEKELKELIEEIEKEIEELEKLKEELKKKLEELEKK
design:0 n:2 mpnn:0.974 plddt:0.954 ptm:0.851 pae:2.626 rmsd:0.881 MELKEELKKEFEEVKKAAKEYLELLEKLKELRPELKEELEKEKEKIEKELKELEEKVKKILESGDEEKAKELIEELKKKKEELKKKIEELKEKIKKIEEE
design:1 n:0 mpnn:1.172 plddt:0.924 ptm:0.828 pae:2.977 rmsd:1.494 SEREREERERRLEEERKKIEELMEKLKKEKDEKKKKELLKELKESVLNRWLLEKGPEAPKVETKEQELEALREELKEAEKKGEKEKAKILEEIIKELEEE
design:1 n:1 mpnn:1.092 plddt:0.844 ptm:0.717

In [8]:
#@title ###Calculation of Rosetta metrics

from pathlib import Path
import pandas as pd

calculate_interface = len(contigs)>1

if chains == '':
  chains=",".join(list('ABCDEFGHIJKLMNOPRSTUVZXYZ'[0:len(contigs)]))

# Is there more than one chain
pyrosetta.init("-corrections::beta_nov16 -mute all")

chain_selector_string=""
for chain in chains.split(","):
  chain_selector_string+=f'\t<Chain name="chain{chain}" chains="{chain}" />\n'

surfaces_filter=""
filter_string_cms= ""
filter_string_sc2= ""
analyze_interface_string=""
mover_interface_string=""
add_mover_interface_string=""
if calculate_interface:
  surfaces_filter+=f'\t<ShapeComplementarity name="sc2" min_sc="0.6" verbose="1" quick="0" residue_selector1="chainA" residue_selector2="chainB" write_int_area="1" write_median_dist="1" confidence="0" />\n'
  surfaces_filter+=f'\t<ContactMolecularSurface name="cms" distance_weight="0.5" use_rosetta_radii="true" apolar_target="0" target_selector="chainA" binder_selector="chainB" confidence="0" />\n'
  filter_string_cms=f'\t<Add filter="cms" />\n'
  filter_string_sc2=f'\t<Add filter="sc2" />\n'
  mover_interface_string+=f'\t<InterfaceAnalyzerMover name="analyze_interface" scorefxn="sfxn_clean"\n'
  mover_interface_string+=f'\tpackstat="1" interface_sc="1" use_jobname="1" jump="1" scorefile_reporting_prefix="IA" />\n'
  add_mover_interface_string=f'\t<Add mover="analyze_interface" />\n'

xml_string="""
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="sfxn_clean" weights="beta_nov16" />
    </SCOREFXNS>

    <RESIDUE_SELECTORS>

"""+chain_selector_string+"""
        <ScoreTermValueBased name="clashing_res" score_type="fa_rep" score_fxn="sfxn"  lower_threshold="3" upper_threshold="99999" />


    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
        <IncludeCurrent name="ic" /> //includes input pdbs rotamers
        <LimitAromaChi2 name="limitaro" chi2max="110" chi2min="70" include_trp="1" /> //disallow extreme aromatic rotamers
        <ExtraRotamersGeneric name="ex1_ex2" ex1="1" ex2="1" /> //use ex1 ex2 rotamers
        <RestrictToRepacking name="repack_only" />  //for minimize/repack
    </TASKOPERATIONS>

    <FILTERS>
        <NetCharge name="chargeA" chain="1" confidence="0" />
        <ExposedHydrophobics name="exposed_hydrop" sasa_cutoff="20" threshold="0" confidence="0"/>

"""+surfaces_filter+"""

        <BuriedUnsatHbonds name="vbuns"  report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" ignore_surface_res="false" print_out_info_to_pdb="true" atomic_depth_selection="5" burial_cutoff="1000" residue_surface_cutoff="42.5" dalphaball_sasa="0" confidence="0"  only_interface="false"  />
        <BuriedUnsatHbonds name="sbuns" report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" cutoff="4" residue_surface_cutoff="42.5" ignore_surface_res="false" print_out_info_to_pdb="true" dalphaball_sasa="0" probe_radius="1.1" atomic_depth_selection="5.5" atomic_depth_deeper_than="false" only_interface="false" confidence="0" />
    </FILTERS>

    <MOVERS>

        <MinMover name="minimize_sc_all" scorefxn="sfxn_clean" bb="0" chi="1" />
"""+mover_interface_string+"""


        <ddG name="ddG_no_repack" translate_by="1000" scorefxn="sfxn_clean" task_operations="repack_only,ic,ex1_ex2" relax_mover="minimize_sc_all"
            repack_bound="0"
            relax_bound="0"
            repack_unbound="0"
            relax_unbound="1"
        jump="1"
        dump_pdbs="0"   />
    </MOVERS>
    <SIMPLE_METRICS>
        <SapScoreMetric name="sap_score" />
        <SelectedResiduesPyMOLMetric name="clashing_res" residue_selector="clashing_res" custom_type="clashing_res" />
    </SIMPLE_METRICS>
    <PROTOCOLS>
        <Add mover="minimize_sc_all" />
"""+add_mover_interface_string+"""
        <Add mover="ddG_no_repack" />
        <Add filter="chargeA" />
        <Add filter="exposed_hydrop" />
        <Add filter="vbuns" />
        <Add filter="sbuns" />
"""+filter_string_cms+"""
        <Add metrics="sap_score" />
        <Add metrics="clashing_res" />
"""+filter_string_sc2+"""
    </PROTOCOLS>
</ROSETTASCRIPTS>

"""
#print(xml_string)
protocol = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects().create_from_string(xml_string).get_mover("ParsedProtocol")

pyrosetta_results=[]
for afile in os.listdir(f"outputs/{path}/all_pdb"):
  design=int(str(Path(afile).stem).split("_")[0][6:])
  idx = int(str(Path(afile).stem).split("_")[1][1:])

  pdb_file = os.path.join(f"outputs/{path}/all_pdb", afile)
  print(pdb_file)
  pose = pyrosetta.pose_from_pdb(pdb_file)
  protocol.apply(pose)
  result = dict(pose.scores)
  result["design"]= design
  result["n"] = idx
  # Some values are returned in an extra dictionary. Merge they keys into one result
  extra_dict = dict(pyrosetta.rosetta.protocols.jd2.get_string_real_pairs_from_current_job())
  if extra_dict:
    for extra_key in  extra_dict.keys():
      if extra_key not in result:
        result[extra_key] = extra_dict[extra_key]
  pyrosetta_results.append(result)

alphafold_df= pd.read_csv(f'outputs/{path}/mpnn_results.csv')
results_df = pd.DataFrame(pyrosetta_results)
merged_df = pd.merge(alphafold_df, results_df, on=['design','n'], how='left')
merged_df.to_csv(f'outputs/{path}/merged_results.csv')




┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python312.ubuntu.cxx11thread.serialization 2025.41+release.de3cc17d509259e29147a2ed8f2a726d644e7e34 2025-10-06T16:25:46] retrieved from: http://www.pyrosetta.org
outputs/test/all_pdb/design0_n0.pdb
outputs/test/all_pdb/design1_n1.pdb
outputs/test/all_pdb/design1_n0.pdb
outputs/test/all_pdb/design0_

In [9]:
# @title ###Show the best designs
from google.colab import data_table
data_table.enable_dataframe_formatter()


interface_fields = [
 'cms',
 'ddg',
 'sc2',
 'sc2_int_area',
 'sc2_median_dist',
 'IA_dSASA_hphobic',
 'IA_dSASA_int',
 'IA_dSASA_polar',
 #'i_ptm,'
]

interesting_fields = '''\
design, n, plddt, rmsd, mpnn, seq, chargeA, sap_score, vbuns, sbuns\
'''.split(', ')
if calculate_interface:
  interesting_fields = interesting_fields + interface_fields
print(interesting_fields)
data_table.DataTable(merged_df[interesting_fields].sort_values("plddt").round(3))


['design', 'n', 'plddt', 'rmsd', 'mpnn', 'seq', 'chargeA', 'sap_score', 'vbuns', 'sbuns']


,design,n,plddt,rmsd,mpnn,seq,chargeA,sap_score,vbuns,sbuns
4,1,1,0.844,2.283,1.092,SAAEERRRREEIERLKREAERLKEEIKKEKDEKKKEELLKRLREAV...,-6.0,17.558,0.0,4.0
3,1,0,0.924,1.494,1.172,SEREREERERRLEEERKKIEELMEKLKKEKDEKKKKELLKELKESV...,-5.0,16.794,0.0,7.0
5,1,2,0.926,1.293,1.176,SAAAEAQRQKEIAELSEKAEELRERIKEEKDEEKREELLRKLREAY...,-15.0,16.419,0.0,9.0
2,0,2,0.954,0.881,0.974,MELKEELKKEFEEVKKAAKEYLELLEKLKELRPELKEELEKEKEKI...,-5.0,24.364,0.0,1.0
1,0,1,0.958,0.767,0.968,MELEKELEEKFKEYKEKAEEVLKLLEELKKKIPEKKEELEAKKKEI...,-5.0,18.893,0.0,3.0
0,0,0,0.975,0.633,1.036,SELEERLKELFKRYEEAVERALALLERLEELRPEEREELEELRREL...,-25.0,22.760,0.0,5.0


In [10]:
#@title ###Display the best result
import py3Dmol
def plot_pdb(design = 0, num = 0):
  hbondCutoff = 4.0
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(f"outputs/{path}/all_pdb/design{design}_n{num}.pdb", 'r').read()
  view.addModel(pdb_str, 'pdb', {'hbondCutoff':hbondCutoff})

  view.setStyle({'cartoon':{'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.addStyle({'stick':{'colorscheme': "GrayCarbon"}})
  view.zoomTo()
  view.show()


design = 0 # @param
num = 0 # @param
plot_pdb(design, num)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
#@title ###Package and download results 📥
#@markdown If you are having issues downloading the result archive,
#@markdown try disabling your adblocker and run this cell again.
#@markdown  If that fails click on the little folder icon to the
#@markdown  left, navigate to file: `name.result.zip`,
#@markdown  right-click and select \"Download\"
#@markdown (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).
!zip -r {path}.result.zip outputs/{path}* outputs/traj/{path}*
files.download(f"{path}.result.zip")

  adding: outputs/test/ (stored 0%)
  adding: outputs/test/merged_results.csv (deflated 46%)
  adding: outputs/test/best_design1.pdb (deflated 78%)
  adding: outputs/test/best.pdb (deflated 78%)
  adding: outputs/test/mpnn_results.csv (deflated 44%)
  adding: outputs/test/all_pdb/ (stored 0%)
  adding: outputs/test/all_pdb/design0_n0.pdb (deflated 78%)
  adding: outputs/test/all_pdb/design1_n1.pdb (deflated 78%)
  adding: outputs/test/all_pdb/design1_n0.pdb (deflated 78%)
  adding: outputs/test/all_pdb/design0_n1.pdb (deflated 78%)
  adding: outputs/test/all_pdb/design0_n2.pdb (deflated 78%)
  adding: outputs/test/all_pdb/design1_n2.pdb (deflated 78%)
  adding: outputs/test/design.fasta (deflated 52%)
  adding: outputs/test/best_design0.pdb (deflated 78%)
  adding: outputs/test_0.pdb (deflated 76%)
  adding: outputs/test_0.trb (deflated 17%)
  adding: outputs/test_1.pdb (deflated 76%)
  adding: outputs/test_1.trb (deflated 16%)
  adding: outputs/traj/test_0_pX0_traj.pdb (deflated 79%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***
***
# **Additional Information: Exploring RFdiffusion Options**

In this tutorial you will explore the different options (`contigs`) of RFdiffusion to generate various structures.  

The `contig` is the most important concept when using RFDiff. It describes which residues are generated and which are kept. The simplest `contig` is just the number of residues to make. `'40/A163-181/40'` says make 40 residues on the N-terminal, keep residues 163-181 of chain A and make 40 new residues on the C-terminal.

Use `contigs` to define continious chains. Use a `:` to define multiple contigs and a `/` to define mutliple segments within a contig.
For example:

**unconditional**
- `contigs='100'` - diffuse **monomer** of length 100
- `contigs='50:100'` - diffuse **hetero-oligomer** of lengths 50 and 100
- `contigs='50'` `symmetry='cyclic'` `order=2` - make two copies of the defined contig(s) and add a symmetry constraint, for **homo-oligomeric** diffusion.

**binder design**
- `contigs='A:50'` `pdb='4N5T'` - diffuse a **binder** of length 50 to chain A of defined PDB.
- `contigs='E6-155:70-100'` `pdb='5KQV'` `hotspot='E64,E88,E96'` - diffuse a **binder** of length 70 to 100 (sampled randomly) to chain E and defined hotspot(s).

**motif scaffolding**
 - `contigs='40/A163-181/50'` `pdb='5TPN'` - add 40 residues to the N-terminal and 50 residues to the C terminal of fragment 161-181 of chain A
 - `contigs='A3-30/20-40/A33-68'` `pdb='6MRR'` - diffuse a loop of length 20 to 40 residues between two segments of defined PDB ranges.

**partial diffusion**
- `contigs=''` `pdb='6MRR'` - noise all coordinates
- `contigs='A1-10'` `pdb='6MRR'` - keep first 10 positions fixed, noise the rest
- `contigs='A'` `pdb='1SSC'` - fix chain A, noise the rest

*hints and tips*
- `pdb=''` leave blank to get an upload prompt
- `contigs='50-100'` use dash to specify a range of lengths to sample from


***
---
#**Acknowledgement** 👤
---
**De novo design of protein structure and function with RFdiffusion**
Watson, Juergens, Bennett, Trippe, Yim, Eisenbach, Aherin ... Baker, Nature 2023

---

RFdiffusion is a method for structure generation, with or without conditional information (a motif, target etc).
RFDiff can perform the following tasks:

*   (unconditional) monomer generation
*   symmetric oligomer generation
*   motif scaffolding
*   symmetric motif scaffolding
*   binder design
*   "partial diffusion", sampling around a design

---

RFdiffusion, ProteinMPNN and AlphaFold parts of this notebook is based on an a notebook by Sergey Ovchinnikov (so3@mit.edu).
https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/rf/examples/diffusion.ipynb

---

Original modifications of the RFdiffusion notebook: Ajasja Ljubetič (ajasja.ljubetic@gmail.com) & Federico Olivieri and with help from the [Ljubetič Lab](https://www.ki.si/en/departments/d12-department-of-synthetic-biology-and-immunology/research-groups/designed-dynamic-proteins/).

---

Original author of RFdiffusion tutorial used here: Ajasja Ljubetič, National Institute of Chemistry (ajasja.ljubetic@gmail.com)

---

Fusion + modification of RFdiffusion, ProteinMPNN and AlphaFold parts for RosettaCon Copenhagen Workshop 2024: M. Ertelt
